In [ ]:
'''import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))''''''

In [ ]:
from IPython.display import clear_output
!pip install  transformers --quiet
!pip install rouge_score -q
!pip install deep-phonemizer -q
!pip install opendatasets -q
!pip install  datasets -q
!pip uninstall -y transformers accelerate -q
!pip install transformers accelerate -q
!pip install evaluate -q
!pip install pycocoevalcap -q
!pip install -U nltk 
!pip install huggingface_hub -q
clear_output()

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
!unzip   /usr/share/nltk_data/corpora/wordnet.zip -d   /usr/share/nltk_data/corpora/



unzip:  cannot find or open /usr/share/nltk_data/corpora/wordnet.zip, /usr/share/nltk_data/corpora/wordnet.zip.zip or /usr/share/nltk_data/corpora/wordnet.zip.ZIP.


In [ ]:
import requests

import os

import datasets
import numpy as np
import pandas as pd
from PIL import Image
from pathlib import Path
#from tqdm import tqdm
#from tqdm.auto import tqdm
#import multiprocessing as mp
import nltk

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import io, transforms
from torch.utils.data import Dataset, DataLoader, random_split

from transformers import Seq2SeqTrainer ,Seq2SeqTrainingArguments
from transformers import VisionEncoderDecoderModel , ViTImageProcessor, ViTFeatureExtractor
from transformers import GPT2Config , default_data_collator
from transformers import GPT2TokenizerFast
from transformers import pipeline,VisionEncoderDecoderConfig

import evaluate

if torch.cuda.is_available():    

    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
'''
from pycocoevalcap.cider.cider import Cider
from pycocoevalcap.spice.spice import Spice '''

'\nfrom pycocoevalcap.cider.cider import Cider\nfrom pycocoevalcap.spice.spice import Spice '

In [ ]:
#Loading data in colab
import opendatasets as od
od.download(
    "https://www.kaggle.com/datasets/adityajn105/flickr8k")

Skipping, found downloaded files in "./flickr8k" (use force=True to force download)


# Initializing the Encoder-Decoder Model

In [ ]:
#encoder_model = "google/vit-base-patch16-224"
encoder_model = "microsoft/swin-base-patch4-window7-224-in22k"
decoder_model = "gpt2"

In [ ]:
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(
    encoder_model, decoder_model).to(device)

Some weights of the model checkpoint at microsoft/swin-base-patch4-window7-224-in22k were not used when initializing SwinModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing SwinModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SwinModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.5.crossattention.masked_bias', 'h.7.ln_cross_attn.weight', 'h.1.crossattention.c_attn.weight', 'h.9.crossattention.masked_bias', 'h.4.crossattention.c_attn.weight', 'h.5.crossattention.q_attn.weight', 'h.0.crossattention.c_attn.weight', 'h.7.crossattention.c_p

In [ ]:
#model.config

## Defining the Tokenizing and Feature Extractor

In [ ]:

tokenizer = GPT2TokenizerFast.from_pretrained(decoder_model)
image_processor =  ViTFeatureExtractor.from_pretrained(encoder_model)

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [ ]:

tokenizer.pad_token = tokenizer.eos_token 
model.config.eos_token_id = tokenizer.eos_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 5
model.config.num_return_sequences = 2
model.config.vocab_size = model.config.decoder.vocab_size


# Preparing the Dataset

In [ ]:
#transformimg the images and converting them to tensors
train_transforms = transforms.Compose(
    [
        transforms.Resize((224,224)), 
        transforms.ToTensor(),
        transforms.Normalize(
            mean=0, 
            std=1
        )
   ]
)


In [ ]:
def load_document(filename):
    file=open(filename,'r',encoding='utf-8')
    text=file.read()
    file.close()
    return text

In [ ]:
import pandas as pd
data=  pd.read_csv("flickr8k/captions.txt")
data

,image,caption
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...
1,1000268201_693b08cb0e.jpg,A girl going into a wooden building .
2,1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .
3,1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playh...
4,1000268201_693b08cb0e.jpg,A little girl in a pink dress going into a woo...
...,...,...
40450,997722733_0cb5439472.jpg,A man in a pink shirt climbs a rock face
40451,997722733_0cb5439472.jpg,A man is rock climbing high in the air .
40452,997722733_0cb5439472.jpg,A person in a red shirt climbing up a rock fac...
40453,997722733_0cb5439472.jpg,A rock climber in a red shirt .


In [ ]:
data = data.groupby('image').apply(lambda x: x.head(3)).reset_index(drop=True)

In [ ]:
data.shape

(24273, 2)

In [ ]:
#train_df , val_df = train_test_split(data , test_size = 0.2)

split_ratio = 0.7
split_index = int(len(data) * split_ratio)

train_df  = data[:split_index]
val_df = data[split_index:]

split_ratio2 = 0.5
split_index2 = int(len(val_df) * split_ratio)

validation_df = val_df[:split_index2]
test_df= val_df[split_index2:]

In [ ]:
class ImgDataset(Dataset):
    def __init__(self, df,root_dir,tokenizer,feature_extractor, transform = None):
        self.df = df
        self.transform = transform
        self.root_dir = root_dir
        self.tokenizer= tokenizer
        self.feature_extractor = feature_extractor
        self.max_length = 32
    def __len__(self,):
        return len(self.df)
    def __getitem__(self,idx):
        caption = self.df.caption.iloc[idx]
        image = self.df.image.iloc[idx]
        img_path = os.path.join(self.root_dir , image)
        img = Image.open(img_path).convert("RGB")
        

        if self.transform is not None:
            img= self.transform(img)
        pixel_values = self.feature_extractor(img, return_tensors="pt").pixel_values.to(device)
        captions = tokenizer(caption, max_length=self.max_length, 
                             padding="max_length", truncation=True).input_ids
        
        encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(captions)}
        return  encoding #{'pixel_values': pixel_values, 'labels':}

In [ ]:
#Colab
train_dataset = ImgDataset(train_df, root_dir = "flickr8k/Images",tokenizer=tokenizer,feature_extractor = image_processor  ,transform = train_transforms)
validation_dataset = ImgDataset(validation_df, root_dir = "flickr8k/Images",tokenizer=tokenizer,feature_extractor = image_processor  ,transform = train_transforms)
test_dataset = ImgDataset(test_df, root_dir = "flickr8k/Images",tokenizer=tokenizer,feature_extractor = image_processor  ,transform = train_transforms)

#Kaggle
'''train_dataset = ImgDataset(train_df, root_dir = "/kaggle/input/flickr8kimagescaptions/flickr8k/images/",tokenizer=tokenizer,feature_extractor = image_processor  ,transform = train_transforms)
validation_dataset = ImgDataset(validation_df, root_dir = "/kaggle/input/flickr8kimagescaptions/flickr8k/images/",tokenizer=tokenizer,feature_extractor = image_processor  ,transform = train_transforms)
test_dataset = ImgDataset(test_df, root_dir = "/kaggle/input/flickr8kimagescaptions/flickr8k/images/",tokenizer=tokenizer,feature_extractor = image_processor  ,transform = train_transforms)
'''

'train_dataset = ImgDataset(train_df, root_dir = "/kaggle/input/flickr8kimagescaptions/flickr8k/images/",tokenizer=tokenizer,feature_extractor = image_processor  ,transform = train_transforms)\nvalidation_dataset = ImgDataset(validation_df, root_dir = "/kaggle/input/flickr8kimagescaptions/flickr8k/images/",tokenizer=tokenizer,feature_extractor = image_processor  ,transform = train_transforms)\ntest_dataset = ImgDataset(test_df, root_dir = "/kaggle/input/flickr8kimagescaptions/flickr8k/images/",tokenizer=tokenizer,feature_extractor = image_processor  ,transform = train_transforms)\n'

In [ ]:
for item in train_dataset:
    print("train_dataset", item["labels"].shape)
    print("train_dataset", item["pixel_values"].shape)
    break
for item in validation_dataset:
    print("validation_dataset", item["labels"].shape)
    print("validation_dataset", item["pixel_values"].shape)
    break
for item in test_dataset:
    print("test_dataset", item["labels"].shape)
    print("test_dataset", item["pixel_values"].shape)
    break

train_dataset torch.Size([32])
train_dataset torch.Size([3, 224, 224])
validation_dataset torch.Size([32])
validation_dataset torch.Size([3, 224, 224])
test_dataset torch.Size([32])
test_dataset torch.Size([3, 224, 224])


# Model Training

## Evaluation Metrics

In [ ]:

rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")
meteor =  evaluate.load("meteor")

def compute_metrics(eval_pred):
    preds = eval_pred.label_ids
    labels = eval_pred.predictions

    pred_str = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels_str = tokenizer.batch_decode(labels, skip_special_tokens=True)

    #rouge score
    rouge_result = rouge.compute(predictions=pred_str, references=labels_str)
    rouge_result =  {k: round(v * 100, 4) for k, v in rouge_result.items()}

    # meteor
    meteor_result = meteor.compute(predictions=pred_str, references=labels_str)

    # bleu scores
    bleu1_result = bleu.compute(predictions=pred_str, references=labels_str, max_order =1)
    bleu2_result = bleu.compute(predictions=pred_str, references=labels_str, max_order =2)
    bleu3_result = bleu.compute(predictions=pred_str, references=labels_str, max_order =3)
    bleu4_result = bleu.compute(predictions=pred_str, references=labels_str, max_order =4)
    
    generation_length = bleu4_result["translation_length"]
    '''#  cider
    # convert lists to dictionaries
    ref_captions = {image_id: [caption] for image_id, caption in enumerate(labels_str)}
    pred_captions = {image_id: [caption] for image_id, caption in enumerate(pred_str)}

    cider_scorer = Cider()
    cider_score, cider_scores = cider_scorer.compute_score(ref_captions, pred_captions)
    #spice
    spice_scorer = Spice()
    spice_score, spice_scores = spice_scorer.compute_score(ref_captions, pred_captions)'''
    return {
      **rouge_result, 
      "meteor": round(meteor_result['meteor']*100, 2) , #round(meteor_result['meteor'], 4),
      "bleu1": round(bleu1_result["bleu"] * 100, 4), #round(bleu1_result["bleu"] , 4), 
      "bleu2": round(bleu2_result["bleu"] * 100, 4), # round(bleu2_result["bleu"], 4), 
      "bleu3": round(bleu3_result["bleu"] * 100, 4), #round(bleu3_result["bleu"] , 4), 
      "bleu4": round(bleu4_result["bleu"]* 100 , 4),#round(bleu4_result["bleu"] , 4)
      #"cider": cider_score,
      #"spice": spice_scores
       #"gen_len": bleu4_result["translation_length"] / len(preds)
      }

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
# collate lists of samples into batches
def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.stack([x['labels'] for x in batch])
    }

In [ ]:
num_epochs = 10
batch_size = 16 

In [ ]:

training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,             
    num_train_epochs=num_epochs,      
    evaluation_strategy="steps", 
    learning_rate = 1.5e-6,
    optim = "adamw_torch",
    eval_steps=2500,                     
    logging_steps=2500,                   
    save_steps=2500,                        
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,  
    output_dir="AsmaMassad/swin-gpt2-image-captioning-flickr8k",
    load_best_model_at_end=True,
    push_to_hub=True # push the model to huggingface hub,
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,                  
    tokenizer=image_processor,    
    args=training_args,              
    compute_metrics=compute_metrics, 
    train_dataset=train_dataset,     
    eval_dataset=validation_dataset,      
    data_collator=collate_fn,        
)

Cloning https://huggingface.co/AsmaMassad/swin-gpt2-image-captioning-flickr8k into local empty directory.


In [ ]:
from torch.utils.data import DataLoader

def get_eval_loader(eval_dataset=None):
    return DataLoader(validation_dataset, collate_fn=collate_fn, batch_size=batch_size)

def get_test_loader(eval_dataset=None):
    return DataLoader(test_dataset, collate_fn=collate_fn, batch_size=batch_size)

trainer.get_train_dataloader = lambda: DataLoader(train_dataset, collate_fn=collate_fn, batch_size=batch_size)
trainer.get_eval_dataloader = get_eval_loader
trainer.get_test_dataloader = get_test_loader

In [ ]:
trainer.train( )

Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Meteor,Bleu1,Bleu2,Bleu3,Bleu4
2500,1.387200,1.178716,20.089000,2.494900,19.057800,19.054200,16.620000,23.583000,7.127900,3.456000,1.965400
5000,1.192200,1.149706,21.872800,3.553000,20.347200,20.338700,13.830000,20.399800,7.708100,3.556800,1.987900
7500,1.165600,1.143236,21.520500,3.940500,20.061500,20.054200,12.430000,17.934800,7.330500,3.483800,1.970900
10000,1.150800,1.139280,22.142100,3.898700,20.563400,20.559400,13.610000,20.256300,7.794400,3.608000,2.040900


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


TrainOutput(global_step=10620, training_loss=1.2198038334675876, metrics={'train_runtime': 12970.6702, 'train_samples_per_second': 13.1, 'train_steps_per_second': 0.819, 'total_flos': 3.0837741499339407e+19, 'train_loss': 1.2198038334675876, 'epoch': 10.0})

In [ ]:
# evaluate on the test_dataset
trainer.evaluate(test_dataset)

{'eval_loss': 1.139279842376709,
 'eval_rouge1': 22.1421,
 'eval_rouge2': 3.8987,
 'eval_rougeL': 20.5634,
 'eval_rougeLsum': 20.5594,
 'eval_meteor': 13.61,
 'eval_bleu1': 20.2563,
 'eval_bleu2': 7.7944,
 'eval_bleu3': 3.608,
 'eval_bleu4': 2.0409,
 'eval_runtime': 565.4312,
 'eval_samples_per_second': 9.014,
 'eval_steps_per_second': 0.564,
 'epoch': 10.0}

In [ ]:
'''beam_outputs = model.generate(
    input_ids, 
    max_length=50, 
    num_beams=5, 
    no_repeat_ngram_size=2, 
    num_return_sequences=5, 
    early_stopping=True
)'''